In [ ]:
import json
import pandas as pd

: 

In [ ]:
# https://apollo.nvu.vsc.edu/classes/remote/lecture_notes/radar/88d/88D_locations.html

# Load the HTML tables into a list of DataFrames
dfs = pd.read_html('blob/88d-locations.html', header=0)
df = dfs[0]

# Rename the columns for easier access
df.columns = ['wban', 'site', 'name', 'coordinate', 'elevation', 'height']

print(df)

In [ ]:
# Raw location in deg/min/sec
def parse_coord_str(string):
    return float(string[:-4]) + float(string[-4:-2]) / 60.0 + float(string[-2:]) / 3600.0

# Raw elevation in feet
def ft2m(ft):
    return ft * 0.3048

# WBAN may be "PENDING"
def parse_int(string):
    try:
        i = int(string)
    except:
        i = None
    return i

In [ ]:
df.set_index('site', inplace=True)
collection = df.to_dict(orient='index')

In [ ]:
new_collection = {}
for key, row in collection.items():
    lat_lon_str = row['coordinate'].split('/')
    lat = parse_coord_str(lat_lon_str[0].strip())
    lon_str = lat_lon_str[1].strip()
    if lon_str[-1] == 'E':
        lon = parse_coord_str(lon_str[:-1])
    else:
        lon = -parse_coord_str(lon_str)
    elevation = ft2m(float(row['elevation']))
    wban = parse_int(row['wban'])
    new_collection[key] = {'latitude': lat, 'longitude': lon, 'elevation': elevation, 'height':float(row['height']), 'wban': wban}

In [ ]:
with open('blob/nexrad-locations.json', 'w') as fid:
    fid.write(json.dumps(new_collection, indent=4))